In [33]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

# old version = python 2
# data = pkl.load(file("ressources/lettres.pkl","rb"))
# new :
with open('../ressources/TME6_lettres.pkl', 'rb') as f:
    data = pkl.load(f, encoding='latin1')
X = np.array(data.get('letters')) # récupération des données sur les lettres
Y = np.array(data.get('labels')) # récupération des étiquettes associées 

nCl = 26

In [49]:
def initGD(X,N):
    S = []
    for x in X:
        S.append(np.floor(np.linspace(0,N-.00000001,len(x))))
    return np.array(S)

In [50]:
def discretise(X,d):
    intervalle = 360/d
    Xd = []
    for x in X:
        Xd.append(np.floor(x/intervalle))
    return np.array(Xd)

In [51]:
Xd = discretise(X,10)
print("X0 =",Xd[0])
print("S0 =",initGD(Xd,4)[0])

X0 = [1. 9. 8. 8. 8. 8. 8. 9. 3. 4. 5. 6. 6. 6. 7. 7. 8. 9. 0. 0. 0. 1. 1.]
S0 = [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3.]


In [127]:
def learnHMM(allx, alls, N, K, initTo0=False):
    if initTo0:
        A = np.zeros((N,N))
        B = np.zeros((N,K))
        Pi = np.zeros(N)
    else:
        eps = 1e-8
        A = np.ones((N,N))*eps
        B = np.ones((N,K))*eps
        Pi = np.ones(N)*eps
    for s in alls:
        Pi[int(s[0])] += 1
        for i in range(len(s)-1):
            A[int(s[i])][int(s[i+1])] += 1 
    for k in range(len(alls)):
        s = alls[k]
        x = allx[k]
        for i in range(len(s)-1):
            B[int(s[i])][int(x[i])] += 1
    A = A/np.maximum(A.sum(1).reshape(N,1),1) # normalisation
    B = B/np.maximum(B.sum(1).reshape(N,1),1) # normalisation
    Pi = Pi/Pi.sum()
    return Pi,A,B

In [128]:
K = 10 # discrétisation (=10 observations possibles)
N = 5  # 5 états possibles (de 0 à 4 en python)
# Xd = angles observés discrétisés
Xd = discretise(X,K)
S = initGD(Xd,N)
Pi, A, B = learnHMM(Xd[Y=='a'],S[Y=='a'],N,K,True)
print("Pi =",Pi)
print("A =\n",A)
print("B =\n",B)

Pi = [1. 0. 0. 0. 0.]
A =
 [[0.79 0.21 0.   0.   0.  ]
 [0.   0.76 0.24 0.   0.  ]
 [0.   0.   0.77 0.23 0.  ]
 [0.   0.   0.   0.76 0.24]
 [0.   0.   0.   0.   1.  ]]
B =
 [[0.06 0.02 0.   0.   0.   0.   0.   0.04 0.49 0.4 ]
 [0.   0.04 0.   0.13 0.09 0.13 0.02 0.09 0.41 0.09]
 [0.   0.   0.   0.02 0.12 0.5  0.31 0.04 0.   0.  ]
 [0.07 0.   0.   0.   0.   0.   0.26 0.33 0.2  0.15]
 [0.71 0.1  0.   0.   0.   0.   0.   0.02 0.02 0.15]]


In [ ]:
def viterbi(x,Pi,A,B):
    delta = np.log(Pi) + np.log(B[i][x[1]])
    phi = -1
    delta